# Task 1: My Model (Custom Model on Tiny-ImageNet-200)

### Import Dependencies and download "Tiny-ImageNet-200" dataset.

In [1]:
# Mounting Google drive for loading the checkpoint.

from google.colab import drive
drive.mount('/content/gdrive')

!cd gdrive/MyDrive/

Mounted at /content/gdrive


In [2]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -q tiny-imagenet-200.zip && ls tiny-imagenet-200

import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor, Lambda
import torchvision.models as models
from torch.hub import load_state_dict_from_url


torch.cuda.empty_cache()
cuda = torch.device('cuda') 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

--2021-11-05 19:45:26--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  31.4MB/s    in 10s     

2021-11-05 19:45:36 (22.9 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]

test  train  val  wnids.txt  words.txt
Using cuda device


### Preprocessing: Collate all image's path address & labels as a input to custom pytorch dataset

In [3]:
file_list = []


for folder in os.listdir('./tiny-imagenet-200/train/'):


  label = folder 
  for file in os.listdir('./tiny-imagenet-200/train/' + folder + '/images/'):
    file_dir = './tiny-imagenet-200/train/' + folder + '/images/' + file

    file_list.append((file_dir))

with open('./tiny-imagenet-200/wnids.txt',) as f:

  id_list = {}
  read_data = f.readlines()
  for i, val in enumerate(read_data):
    id_list[val.replace('\n', '')] = i


test_list = []
test_id = {}
with open('./tiny-imagenet-200/val/val_annotations.txt', 'r') as f:
  for line in f.readlines():
    file, label = line.split()[0:2]
    file_dir = './tiny-imagenet-200/val/images/' + file

    test_list.append((file_dir))
    test_id[file] = label

### Custom Pytorch datatset for training images of "Tiny-ImageNet-200" dataset.

In [4]:
class TrainTinyImageNetDataset(Dataset):
    def __init__(self, f_list, id, transform=None):

        self.filenames = f_list
        self.transform = transform
        self.id_dict = id

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):

        img_path = self.filenames[idx]
        image = None
       
        with open(img_path, 'rb') as f:
          image = Image.open(f)
          image =  image.convert('RGB')
          
        
       

        label = self.id_dict[img_path.split('/')[-1].split('.')[0].split('_')[0]]
       

        if self.transform is not None:

            image = self.transform(image)
            
        return image, label


class TrainSet(TrainTinyImageNetDataset):
  
    def __init__(self, f_list, id, transform=None):

        super(TrainSet, self).__init__(f_list, id, transform=transform)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


### Custom Pytorch datatset for testing images of "Tiny-ImageNet-200" dataset.

In [5]:
class TestTinyImageNetDataset(Dataset):
    def __init__(self, t_list, id, cls_id, transform=None):
        self.filenames = t_list
        self.transform = transform
        self.id_dict = id
        self.cls_id = cls_id
       


    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_path = self.filenames[idx]
        image = None
       
        with open(img_path, 'rb') as f:
          image = Image.open(f)
          image =  image.convert('RGB')
    
        label = self.cls_id[self.id_dict[img_path.split('/')[-1]]]
        
        if self.transform is not None:
            image = self.transform(image)
        return image, label
       

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


### Defining "MyModel" (Custom Model for Task-1)

### This model contain 7 Convolution Blocks (with Nomral Convolution, Batch Normalization & RELU Block Layers)

### It also contains 4 Upsampling layers, 3 Skip-connections & a 2 layer classification head

In [6]:
# My Model

class MyModel(nn.Module):
    expansion = 1

    def __init__(self, stride=1, downsample=None):
        super(MyModel, self).__init__()
        
        # Nomral Convolution, Batch Normalization & RELU Block Layers for 7 such blocks
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3)
        self.bn4 = nn.BatchNorm2d(128)
        self.conv5 = nn.Conv2d(128, 128, kernel_size=3)
        self.bn5 = nn.BatchNorm2d(128)
        self.conv6 = nn.Conv2d(128, 256, kernel_size=3)
        self.bn6 = nn.BatchNorm2d(256)
        self.conv7 = nn.Conv2d(256, 256, kernel_size=3)
        self.bn7 = nn.BatchNorm2d(256)
        self.conv7 = nn.Conv2d(256, 512, kernel_size=3)
        self.bn7 = nn.BatchNorm2d(512)


        # Convolution layer for reducing the skip-connections to appropriate channel dimensions
        self.out1_conv = nn.Conv2d(64, 128, kernel_size=1)
        self.out2_conv = nn.Conv2d(128, 256, kernel_size=1)
        self.out3_conv = nn.Conv2d(256, 512, kernel_size=1)


        # Last FC & Classification layers
        self.fc1 = nn.Linear(100352, 4096)
        self.fc2 = nn.Linear(4096, 200)

        # Upsampling layers
        self.up1 = nn.ConvTranspose2d(64, 64, 2, stride=2)
        self.up2 = nn.ConvTranspose2d(128, 128, 2, stride=2)
        self.up3 = nn.ConvTranspose2d(256, 256, 2, stride=2)
        self.up4 = nn.ConvTranspose2d(512, 512, 2, stride=2)

        # Pooling laye
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        
  
       

    def forward(self, x):
        identity = x

        # 1st CONV block
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.avgpool(out)
        out1 = out
        
        # 1st Reducing Layer
        out1 = self.out1_conv(out1)

        # 1st Upsampling
        out = self.up1(out)
        
        # 2nd CONV block
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.avgpool(out)

        # 3rd CONV block
        out = self.conv3(out)
        out = self.bn3(out)
        out = self.relu(out)
        out = self.avgpool(out)

        # 4th CONV block
        out = self.conv4(out)
        out = self.bn4(out)
        out = self.relu(out)
        out = self.avgpool(out)

        # 1st skip-connection 
        out = out + out1
        out2 = out

        # 2nd Reducing Layer
        out2 = self.out2_conv(out2)

        # 2nd Upsampling
        out = self.up2(out)
      
        # 5th CONV block
        out = self.conv5(out)
        out = self.bn5(out)
        out = self.relu(out)
        out = self.avgpool(out)

        # # 6th CONV block
        out = self.conv6(out)
        out = self.bn6(out)
        out = self.relu(out)
        out = self.avgpool(out)

        # 2nd skip-connection
        out = out + out2
        out3 = out

        # 3rd Reducing Layer
        out3 = self.out3_conv(out3)


        # 3rd Upsampling
        out = self.up3(out)
        
        # 7th CONV block
        out = self.conv7(out)
        out = self.bn7(out)
        out = self.relu(out)
        out = self.avgpool(out)

        # 3rd skip-connection
        out = out + out3

        # 4th Upsampling
        out = self.up4(out)
        
       
        # Flatten
        out = torch.flatten(out, 1)

        # Last FC & Classification layers
        out = self.fc1(out)
        out = self.fc2(out)

        return out




### Training Function

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
      
        X, y = X.to(device), y.long().to(device)
        optimizer.zero_grad()
        torch.set_grad_enabled(True)
        
        pred = model(X).float()
       
        loss = loss_fn(pred, y)

       
        loss.backward()
        optimizer.step()

    
        if batch % 1000 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


### Testing Function

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
          
            X, y = X.to(device), y.long().to(device)
            torch.set_grad_enabled(False)
            pred = model(X).float()
       
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss

### Defining "Learning Rate Scheduler" & "Early stopping mechanism"

In [9]:
class LRScheduler():
  
    def __init__(
        self, optimizer, patience=5, min_lr=1e-7, factor=0.75
    ):
        
        self.optimizer = optimizer
        self.patience = patience
        self.min_lr = min_lr
        self.factor = factor
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau( 
                self.optimizer,
                mode='min',
                patience=self.patience,
                factor=self.factor,
                min_lr=self.min_lr,
                verbose=True
            )
    def __call__(self, val_loss):
        self.scheduler.step(val_loss)


class EarlyStopping():
   
    def __init__(self, patience=10, min_delta=0):
      
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss == None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        elif self.best_loss - val_loss < self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

### Instantiating Dataset loaders for training & testing

In [11]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

trans = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])

trainset = TrainSet(f_list=file_list,id=id_list, transform=trans)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=5, shuffle=True, num_workers=4)

trans_test = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            normalize,
        ])

testset = TestTinyImageNetDataset(t_list=test_list,id=test_id, cls_id =id_list,  transform=trans_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=5, shuffle=False, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


### Instantiating & training & checkpoint "MyModel"

#### There are 2 cells for "MyModel" that are given below.
#### Run the 1st cell in case you want to load a checkpoint & resume training.
#### Else Run the 2nd cell in case you want to start training from scratch.

#### **NOTE: Make sure to change the path of loading the checkpoint in 1st cell**

In [13]:
MyModel = MyModel()
MyModel = MyModel.to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer_MyModel = torch.optim.Adam(MyModel.parameters(), lr=0.0001)

# **NOTE: Make sure to change the path of loading the checkpoint in 1st cell**
checkpoint = torch.load('gdrive/MyDrive/trained-MyModel-model.ckpt')
MyModel.load_state_dict(checkpoint['model_state_dict'])
optimizer_MyModel.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch'] + 1
loss = checkpoint['loss']

es =  EarlyStopping()
lrs = LRScheduler(optimizer_MyModel)


checkpoint_dir = "."
test_loss = 0
epochs = 20

for i in range(epochs):
    print(f"Epoch {i+1}")
    train(trainloader, MyModel, loss_fn, optimizer_MyModel)
    test_loss = test(testloader, MyModel, loss_fn)

    torch.save({
        'epoch': i+1,
        'model_state_dict': MyModel.state_dict(),
        'optimizer_state_dict': optimizer_MyModel.state_dict(),
        'loss': test_loss
        }, checkpoint_dir+'/%04d-MyModel-model.ckpt' %i)
    
    lrs(test_loss)
    es(test_loss)
    if es.early_stop: 
         break

Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


loss: 7.514592  [    0/100000]
loss: 1.700148  [ 5000/100000]
loss: 1.500712  [10000/100000]
loss: 3.206714  [15000/100000]
loss: 2.439969  [20000/100000]
loss: 3.403428  [25000/100000]
loss: 0.760292  [30000/100000]
loss: 2.898070  [35000/100000]
loss: 4.183686  [40000/100000]
loss: 3.057768  [45000/100000]
loss: 3.208254  [50000/100000]
loss: 2.270092  [55000/100000]
loss: 2.450874  [60000/100000]
loss: 3.304353  [65000/100000]
loss: 3.263307  [70000/100000]
loss: 5.986336  [75000/100000]
loss: 2.825410  [80000/100000]
loss: 3.887670  [85000/100000]
loss: 2.100561  [90000/100000]
loss: 1.929519  [95000/100000]
Test Error: 
 Accuracy: 26.6%, Avg loss: 3.347503 

Epoch 2
loss: 2.536825  [    0/100000]
loss: 2.335321  [ 5000/100000]
loss: 2.697928  [10000/100000]
loss: 1.823586  [15000/100000]
loss: 2.413153  [20000/100000]
loss: 1.476184  [25000/100000]
loss: 2.381083  [30000/100000]
loss: 4.326852  [35000/100000]
loss: 3.465562  [40000/100000]
loss: 4.179102  [45000/100000]
loss: 3.09

KeyboardInterrupt: ignored

#### Run the 2nd cell in case you want to start training from scratch.

In [10]:
MyModel = MyModel()
MyModel = MyModel.to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer_MyModel = torch.optim.Adam(MyModel.parameters(), lr=0.0001)


es =  EarlyStopping()
lrs = LRScheduler(optimizer_MyModel)

checkpoint_dir = "."
test_loss = 0
epochs = 20

for i in range(epochs):
    print(f"Epoch {i+1}")
    train(trainloader, MyModel, loss_fn, optimizer_MyModel)
    test_loss = test(testloader, MyModel, loss_fn)

    torch.save({
        'epoch': i+1,
        'model_state_dict': MyModel.state_dict(),
        'optimizer_state_dict': optimizer_MyModel.state_dict(),
        'loss': test_loss
        }, checkpoint_dir+'/%04d-MyModel-model.ckpt' %i)
    
    lrs(test_loss)
    es(test_loss)
    if es.early_stop: 
         break



Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


loss: 5.311528  [    0/100000]
loss: 5.459359  [ 5000/100000]
loss: 4.884769  [10000/100000]
loss: 4.424979  [15000/100000]
loss: 5.160327  [20000/100000]
loss: 4.749619  [25000/100000]
loss: 3.338456  [30000/100000]
loss: 4.676524  [35000/100000]
loss: 5.065419  [40000/100000]
loss: 5.037518  [45000/100000]
loss: 4.342889  [50000/100000]
loss: 4.203567  [55000/100000]
loss: 4.501772  [60000/100000]
loss: 4.284214  [65000/100000]
loss: 4.768407  [70000/100000]
loss: 4.048433  [75000/100000]
loss: 3.480863  [80000/100000]
loss: 4.337773  [85000/100000]
loss: 4.127688  [90000/100000]
loss: 4.351015  [95000/100000]
Test Error: 
 Accuracy: 12.6%, Avg loss: 4.222650 

Epoch 2
loss: 3.784712  [    0/100000]
loss: 3.439715  [ 5000/100000]
loss: 3.591293  [10000/100000]
loss: 3.580187  [15000/100000]
loss: 4.193014  [20000/100000]
loss: 4.775201  [25000/100000]
loss: 3.773143  [30000/100000]
loss: 3.888266  [35000/100000]
loss: 2.206846  [40000/100000]
loss: 3.233969  [45000/100000]
loss: 5.49

KeyboardInterrupt: ignored